### Automatic stock filter analysis + performance backtest logging

In [6]:
import coremlv2 as core
import importlib
core._init_ml()
importlib.reload(core)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
# Limiting GPU memory growth
gpus = core.tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            core.tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = core.tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs, ", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs,  1 Logical GPUs


In [3]:
# Load Model
# OPEN MODEL (whole-data-traning_11, epochs-subepochs)
# 0.4 vid_constituents
model_source = '327-3_cl-0.5_idc-1_mid-0.4_ep-5_bz-1024_ldtv-2_sbuffs-1_gen-1_1-4'

# 0.5 vid_constituents
model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-5_bz-2048_ldtv-2_sbuffs-1_gen-1_4-16'
model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-5_bz-2048_ldtv-2_sbuffs-1_gen-1_4-19'
model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-15_bz-2048_ldtv-2_sbuffs-1_gen-1_0-7'
model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-15_bz-2048_ldtv-2_sbuffs-1_gen-1_1-3'
model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-15_bz-2048_ldtv-2_sbuffs-1_gen-1_1-7'
model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-15_bz-2048_ldtv-2_sbuffs-1_gen-1_1-16'
model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-15_bz-2048_ldtv-2_sbuffs-1_gen-1_2-7'
model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-15_bz-2048_ldtv-2_sbuffs-1_gen-1_4-3'
model_source = '327-6ds_cl-0.05_idc-1_mid-0.5_ep-40_bz-1536_ldtv-3_sbuffs-1_gen-1_1-0' # 0.6681 / 0.5908
model_source = '327-6ds_cl-0.1_idc-1_mid-0.5_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_1-4' # 0.6627 / 0.5987
model_source = '327-9_cl-0.001_idc-0.25_mid-0.9_ep-40_bz-512_ldtv-3_sbuffs-1_gen-1_0-0' # 0.7098 / 0.5346
model_source = '327-9_cl-0.005_idc-1_mid-0.9_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_28-0' # 0.6459 / 0.6173
model_source = '327-9_cl-0.15_idc-1_mid-0.9_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_3-5' # 0.6334 / 0.6327
model_source = '327-9_cl-0.15_idc-1_mid-0.9_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_7-1' # 0.6298 / 0.6349
model_source = '327-9_cl-0.35_idc-1_mid-0.8_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_0-1' # 0.6176 / 0.6467

ROOT_PATH = './'

model_source_path = core.os.path.join(ROOT_PATH, f'models/preloaded/{model_source}')
model = core.tf.keras.models.load_model(f'{model_source_path}/model')

In [4]:
# Safe dataset list
dataset = 'vid'
db_ver = '8'
DB_ROOT_PATH = 'E:\#PROJECT\idx'
model_no = '327'
sdl = core.safe_dataset_list(dataset, db_ver=db_ver, DB_ROOT_PATH=DB_ROOT_PATH, return_full_filename=True, model_no=model_no)

In [5]:
# Output file
outfile = model_source + '.json'
outfile_path = core.os.path.join(ROOT_PATH, 'backtest_performance_ratio')
outfile_path = core.os.path.join(outfile_path, outfile)
if not core.os.path.exists(outfile_path):
    # Initialize blank variable
    backtest_results = {}
elif core.os.path.exists(outfile_path):
    # Load current progress
    with open(outfile_path, 'r') as f:
        backtest_results = core.json.load(f)    
    
for i, dataset in enumerate(sdl):
    # Hotstart
    if dataset[:7] in backtest_results:
        print(f'{i+1}/{len(sdl)} Continue {dataset}...')
        continue
    else:
        print(f'{i+1}/{len(sdl)} Processing {dataset}...')
    
    backtest_results[dataset[:7]] = {}
    # Load v2 dataset to test wheter the model is valid or not with newer db version
    db_path = core.os.path.join(DB_ROOT_PATH, 'db')
    db_path = core.os.path.join(db_path, 'v8')
    db_path = core.os.path.join(db_path, 'dataset')
    db_path = core.os.path.join(db_path, 'vid')
    db_path = core.os.path.join(db_path, dataset)
    with core.h5py.File(db_path, 'r') as f:
        train_inputs = f['train_inputs'][()]
        test_inputs = f['test_inputs'][()]
        train_labels = f['train_labels'][()]
        train_changes = f['train_changes'][()]
        test_labels = f['test_labels'][()]
        test_changes = f['test_changes'][()]

    train_inputs = core.np.nan_to_num(train_inputs, posinf=0.0, neginf=0.0)
    test_inputs = core.np.nan_to_num(test_inputs, posinf=0.0, neginf=0.0)
    
    try:
        # Evaluate overall performance
        evaluate_train = model.evaluate(train_inputs, train_labels, verbose=0)
        evaluate_test = model.evaluate(test_inputs, test_labels, verbose=0)
        backtest_results[dataset[:7]]['train_loss'] = evaluate_train[0]
        backtest_results[dataset[:7]]['train_accuracy'] = evaluate_train[1]
        backtest_results[dataset[:7]]['test_loss'] = evaluate_test[0]
        backtest_results[dataset[:7]]['test_accuracy'] = evaluate_test[1]

        # Backtest performance
        train_ideal, train_real, test_ideal, test_real = core.backtest_v1(model, train_inputs, test_inputs, train_labels, test_labels, train_changes, test_changes)
        backtest_results[dataset[:7]]['train_ideal'] = list(train_ideal[0])
        backtest_results[dataset[:7]]['train_real'] = list(train_real[0])
        backtest_results[dataset[:7]]['test_ideal'] = list(test_ideal[0])
        backtest_results[dataset[:7]]['test_real'] = list(test_real[0])

        # Performance ratio
        train_r, train_nr, test_r, test_nr = core.performance_ratio(train_ideal, train_real, test_ideal, test_real)
        backtest_results[dataset[:7]]['train_positive_ratio'] = train_r
        backtest_results[dataset[:7]]['train_negative_ratio'] = train_nr
        backtest_results[dataset[:7]]['test_positive_ratio'] = test_r
        backtest_results[dataset[:7]]['test_negative_ratio'] = test_nr
    except:
        # Evaluate overall performance
        backtest_results[dataset[:7]]['train_loss'] = 0
        backtest_results[dataset[:7]]['train_accuracy'] = 0
        backtest_results[dataset[:7]]['test_loss'] = 0
        backtest_results[dataset[:7]]['test_accuracy'] = 0

        # Backtest performance
        backtest_results[dataset[:7]]['train_ideal'] = [0, 0]
        backtest_results[dataset[:7]]['train_real'] = [0, 0]
        backtest_results[dataset[:7]]['test_ideal'] = [0, 0]
        backtest_results[dataset[:7]]['test_real'] = [0, 0]

        # Performance ratio
        backtest_results[dataset[:7]]['train_positive_ratio'] = 0
        backtest_results[dataset[:7]]['train_negative_ratio'] = 0
        backtest_results[dataset[:7]]['test_positive_ratio'] = 0
        backtest_results[dataset[:7]]['test_negative_ratio'] = 0
    
    # Save current progress to file
    with open(outfile_path, 'w') as f:
        core.json.dump(backtest_results, f)

1/423 Processing AALI.JK_shift0_interval5_recurrent120_split0.8.hdf5...
2/423 Processing ABMM.JK_shift0_interval5_recurrent120_split0.8.hdf5...
3/423 Processing ACES.JK_shift0_interval5_recurrent120_split0.8.hdf5...
4/423 Processing ACST.JK_shift0_interval5_recurrent120_split0.8.hdf5...
5/423 Processing ADES.JK_shift0_interval5_recurrent120_split0.8.hdf5...
6/423 Processing ADHI.JK_shift0_interval5_recurrent120_split0.8.hdf5...
7/423 Processing ADMF.JK_shift0_interval5_recurrent120_split0.8.hdf5...
8/423 Processing ADRO.JK_shift0_interval5_recurrent120_split0.8.hdf5...
9/423 Processing AGII.JK_shift0_interval5_recurrent120_split0.8.hdf5...
10/423 Processing AGRO.JK_shift0_interval5_recurrent120_split0.8.hdf5...
11/423 Processing AGRS.JK_shift0_interval5_recurrent120_split0.8.hdf5...
12/423 Processing AHAP.JK_shift0_interval5_recurrent120_split0.8.hdf5...
13/423 Processing AKKU.JK_shift0_interval5_recurrent120_split0.8.hdf5...
14/423 Processing AKRA.JK_shift0_interval5_recurrent120_spli

C:\Users\jonat\Documents\#PROJECT\idx\coremlv2.py:2404: RuntimeWarning: invalid value encountered in double_scalars
  test_r = test_real[0][0] / test_ideal[0][0]
C:\Users\jonat\Documents\#PROJECT\idx\coremlv2.py:2405: RuntimeWarning: invalid value encountered in double_scalars
  test_nr = test_real[0][1] / test_ideal[0][1]


22/423 Processing ANJT.JK_shift0_interval5_recurrent120_split0.8.hdf5...
23/423 Processing ANTM.JK_shift0_interval5_recurrent120_split0.8.hdf5...
24/423 Processing APIC.JK_shift0_interval5_recurrent120_split0.8.hdf5...
25/423 Processing APII.JK_shift0_interval5_recurrent120_split0.8.hdf5...
26/423 Processing APLN.JK_shift0_interval5_recurrent120_split0.8.hdf5...
27/423 Processing ARII.JK_shift0_interval5_recurrent120_split0.8.hdf5...
28/423 Processing ARKA.JK_shift0_interval5_recurrent120_split0.8.hdf5...
29/423 Processing ARTA.JK_shift0_interval5_recurrent120_split0.8.hdf5...
30/423 Processing ARTO.JK_shift0_interval5_recurrent120_split0.8.hdf5...
31/423 Processing ASGR.JK_shift0_interval5_recurrent120_split0.8.hdf5...
32/423 Processing ASII.JK_shift0_interval5_recurrent120_split0.8.hdf5...
33/423 Processing ASMI.JK_shift0_interval5_recurrent120_split0.8.hdf5...
34/423 Processing ASRI.JK_shift0_interval5_recurrent120_split0.8.hdf5...
35/423 Processing ASSA.JK_shift0_interval5_recurren

C:\Users\jonat\Documents\#PROJECT\idx\coremlv2.py:2402: RuntimeWarning: invalid value encountered in double_scalars
  train_r = train_real[0][0] / train_ideal[0][0]


242/423 Processing MAPA.JK_shift0_interval5_recurrent120_split0.8.hdf5...
243/423 Processing MAPB.JK_shift0_interval5_recurrent120_split0.8.hdf5...
244/423 Processing MAPI.JK_shift0_interval5_recurrent120_split0.8.hdf5...
245/423 Processing MARI.JK_shift0_interval5_recurrent120_split0.8.hdf5...
246/423 Processing MARK.JK_shift0_interval5_recurrent120_split0.8.hdf5...
247/423 Processing MASA.JK_shift0_interval5_recurrent120_split0.8.hdf5...
248/423 Processing MAYA.JK_shift0_interval5_recurrent120_split0.8.hdf5...
249/423 Processing MBAP.JK_shift0_interval5_recurrent120_split0.8.hdf5...
250/423 Processing MBSS.JK_shift0_interval5_recurrent120_split0.8.hdf5...
251/423 Processing MBTO.JK_shift0_interval5_recurrent120_split0.8.hdf5...
252/423 Processing MCAS.JK_shift0_interval5_recurrent120_split0.8.hdf5...
253/423 Processing MCOR.JK_shift0_interval5_recurrent120_split0.8.hdf5...
254/423 Processing MDIA.JK_shift0_interval5_recurrent120_split0.8.hdf5...
255/423 Processing MDKI.JK_shift0_inte

C:\Users\jonat\Documents\#PROJECT\idx\coremlv2.py:2403: RuntimeWarning: invalid value encountered in double_scalars
  train_nr = train_real[0][1] / train_ideal[0][1]


393/423 Processing TRIS.JK_shift0_interval5_recurrent120_split0.8.hdf5...
394/423 Processing TRST.JK_shift0_interval5_recurrent120_split0.8.hdf5...
395/423 Processing TRUK.JK_shift0_interval5_recurrent120_split0.8.hdf5...
396/423 Processing TRUS.JK_shift0_interval5_recurrent120_split0.8.hdf5...
397/423 Processing TSPC.JK_shift0_interval5_recurrent120_split0.8.hdf5...
398/423 Processing TUGU.JK_shift0_interval5_recurrent120_split0.8.hdf5...
399/423 Processing TURI.JK_shift0_interval5_recurrent120_split0.8.hdf5...
400/423 Processing UNSP.JK_shift0_interval5_recurrent120_split0.8.hdf5...
401/423 Processing UNTR.JK_shift0_interval5_recurrent120_split0.8.hdf5...
402/423 Processing UNVR.JK_shift0_interval5_recurrent120_split0.8.hdf5...
403/423 Processing URBN.JK_shift0_interval5_recurrent120_split0.8.hdf5...
404/423 Processing VICO.JK_shift0_interval5_recurrent120_split0.8.hdf5...
405/423 Processing VINS.JK_shift0_interval5_recurrent120_split0.8.hdf5...
406/423 Processing VIVA.JK_shift0_inte

#### Stock with positive ratio

In [7]:
model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-15_bz-2048_ldtv-2_sbuffs-1_gen-1_0-7'
# model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-15_bz-2048_ldtv-2_sbuffs-1_gen-1_1-3'
# model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-15_bz-2048_ldtv-2_sbuffs-1_gen-1_1-7'
# model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-15_bz-2048_ldtv-2_sbuffs-1_gen-1_1-16'
# model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-15_bz-2048_ldtv-2_sbuffs-1_gen-1_2-7'
# model_source = '327-3_cl-0.5_idc-1_mid-0.5_ep-15_bz-2048_ldtv-2_sbuffs-1_gen-1_4-3'
model_source = '327-6ds_cl-0.05_idc-1_mid-0.5_ep-40_bz-1536_ldtv-3_sbuffs-1_gen-1_1-0'
model_source = '327-9_cl-0.001_idc-0.25_mid-0.9_ep-40_bz-512_ldtv-3_sbuffs-1_gen-1_0-0'
# model_source = '327-9_cl-0.005_idc-1_mid-0.9_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_28-0'
# model_source = '327-9_cl-0.15_idc-1_mid-0.9_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_3-5'
model_source = '327-9_cl-0.35_idc-1_mid-0.8_ep-40_bz-2048_ldtv-3_sbuffs-1_gen-1_0-1' # 0.6176 / 0.6467

ROOT_PATH = './'

# Output file
outfile = model_source + '.json'
outfile_path = core.os.path.join(ROOT_PATH, 'backtest_performance_ratio')
outfile_path = core.os.path.join(outfile_path, outfile)

In [8]:
with open(outfile_path, 'r') as f:
    backtest_results = core.json.load(f)

In [9]:
backtest_results_df = core.pd.DataFrame(backtest_results)
# backtest_results_df = backtest_results_df[['train_loss', 'train_accuracy', 'test_loss', 'test_accuracy', 'train_positive_ratio', 'train_negative_ratio', 'test_positive_ratio', 'test_negative_ratio']]
# backtest_results_df.drop(['train_ideal','train_real','test_ideal','test_real'], axis=0)
backtest_results_df = backtest_results_df.T
backtest_results_df.dropna(subset=['train_positive_ratio'], inplace=True)
backtest_results_df.dropna(subset=['train_negative_ratio'], inplace=True)
backtest_results_df.dropna(subset=['test_positive_ratio'], inplace=True)
backtest_results_df.dropna(subset=['test_negative_ratio'], inplace=True)

In [10]:
backtest_results_df.mean()

C:\Users\jonat\AppData\Local\Temp\ipykernel_16992\37365717.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  backtest_results_df.mean()


train_loss              0.624687
train_accuracy          0.640170
test_loss               0.667935
test_accuracy           0.589971
train_positive_ratio    0.113170
train_negative_ratio   -0.037960
test_positive_ratio    -0.107664
test_negative_ratio     1.480929
dtype: float64

In [9]:
backtest_results_df.mean()

C:\Users\jonat\AppData\Local\Temp\ipykernel_20352\37365717.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  backtest_results_df.mean()


train_loss              0.639690
train_accuracy          0.625679
test_loss               0.663829
test_accuracy           0.595289
train_positive_ratio    0.070119
train_negative_ratio   -0.092837
test_positive_ratio    -0.034890
test_negative_ratio     1.563569
dtype: float64

In [19]:
backtest_results_df.mean()

C:\Users\jonat\AppData\Local\Temp\ipykernel_20352\37365717.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  backtest_results_df.mean()


train_loss              0.650818
train_accuracy          0.610871
test_loss               0.671438
test_accuracy           0.585596
train_positive_ratio    0.047612
train_negative_ratio   -0.131034
test_positive_ratio     0.034273
test_negative_ratio     1.251621
dtype: float64

In [24]:
backtest_results_df.mean()

C:\Users\jonat\AppData\Local\Temp\ipykernel_20352\37365717.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  backtest_results_df.mean()


train_loss              0.706042
train_accuracy          0.538161
test_loss               0.727713
test_accuracy           0.522759
train_positive_ratio    0.000631
train_negative_ratio   -0.208250
test_positive_ratio     0.017256
test_negative_ratio          inf
dtype: float64

In [20]:
backtest_results_df.loc['AALI.JK']

train_loss                                               0.643234
train_accuracy                                            0.62489
test_loss                                                0.672679
test_accuracy                                            0.594737
train_ideal              [49.40660803376498, -31.905175109775307]
train_real                [16.75474834779059, 0.7466845761990775]
test_ideal                 [9.63076326278257, -8.850747274451718]
test_real               [1.6374564880481035, -0.8574404997172482]
train_positive_ratio                                      0.33912
train_negative_ratio                                    -0.023403
test_positive_ratio                                      0.170024
test_negative_ratio                                      0.096878
Name: AALI.JK, dtype: object

In [14]:
# (backtest_results_df['train_positive_ratio'] > 0) & (backtest_results_df['train_negative_ratio'] > 0) & 
positive_ratios = backtest_results_df.loc[(backtest_results_df['train_positive_ratio'] > 0) & (backtest_results_df['train_negative_ratio'] > 0) & (backtest_results_df['test_positive_ratio'] > 0) & (backtest_results_df['test_negative_ratio'] > 0)]

In [15]:
positive_ratios.sort_values(by='test_loss', ascending=True)

,train_loss,train_accuracy,test_loss,test_accuracy,train_ideal,train_real,test_ideal,test_real,train_positive_ratio,train_negative_ratio,test_positive_ratio,test_negative_ratio
DEWA.JK,0.458566,0.812913,0.294307,0.993243,"[22.602662817241438, -24.13935934872159]","[2.8716873071253577, -4.408383838605521]","[0.5483870483059123, -0.2885332764212861]","[0.7737065857374399, -0.5138528138528138]",0.127051,0.182622,1.410877,1.780914
SWAT.JK,0.66982,0.574675,0.585769,0.8125,"[5.796478703749098, -6.7215885387200185]","[1.5327452838416846, -2.4578551188126077]","[0.788618753824017, -1.3250260437671624]","[0.4091755362236994, -0.9455828261668447]",0.264427,0.365666,0.518851,0.713633
FOOD.JK,0.661318,0.584,0.615241,0.722222,"[9.892598143973553, -9.165120901857433]","[3.3968296496762274, -2.6693524075601074]","[0.4789310408212634, -0.41759524432755185]","[0.5245300232240049, -0.4631942267302933]",0.343371,0.291251,1.09521,1.109194
YPAS.JK,0.587292,0.672112,0.62842,0.660808,"[37.122561341383076, -26.464510200736605]","[14.90420490277252, -4.246153762126058]","[26.461811953073248, -14.687900967123603]","[14.436919483059295, -2.6630084971096544]",0.401486,0.160447,0.545576,0.181306
KOIN.JK,0.608046,0.687357,0.633255,0.633157,"[57.066180236844595, -36.985456795244254]","[22.52073904947347, -2.4400156078731268]","[12.212226554866236, -7.8273227674999495]","[7.760968449622241, -3.376064662255953]",0.394642,0.065972,0.635508,0.431318
MAPB.JK,0.661822,0.592953,0.641091,0.655172,"[6.95162033209899, -7.837527167718616]","[2.4078120596786663, -3.293718895298294]","[1.4395509866343867, -1.1533367118037394]","[0.8895619994541086, -0.6033477246234612]",0.346367,0.42025,0.617944,0.523132
MLPT.JK,0.630562,0.653266,0.64242,0.649351,"[26.983679681613882, -23.18279062834967]","[5.074388170699001, -1.2734991174347907]","[12.709456428919214, 0.31419827140933965]","[2.505029172099816, 10.518625528228737]",0.188054,0.054933,0.1971,33.477668
VRNA.JK,0.655259,0.598453,0.643956,0.644524,"[36.15460630239962, -26.550055865137313]","[9.864851129965132, -0.2603006927028325]","[17.514578616948747, -11.060953409446952]","[7.77558599477772, -1.3219607872759314]",0.272852,0.009804,0.443949,0.119516
TRST.JK,0.675354,0.579475,0.646153,0.646438,"[47.171835456000196, -36.22475348016419]","[12.431981668871465, -1.484899693035458]","[8.177102667033704, -5.62366599520192]","[2.787424008939558, -0.23398733710777392]",0.263547,0.040991,0.340882,0.041608
ARII.JK,0.640522,0.624286,0.650706,0.625,"[26.942647120476018, -24.44355461490377]","[8.857564191185537, -6.358471685613284]","[6.630069612718829, -7.203608292730864]","[0.6545037992640449, -1.228042479276081]",0.328756,0.260129,0.098717,0.170476
